# Introduction to Obspy

### M2 - UFAZ - Seismological modeling
@D. Zigone, J. Vergne, A. Maggi / Nov. 2021

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Introduction

Seismological data can be provided in a variety of formats (SAC, miniseed, segY, segD, wav, etc.). The format that is curently the most wedeli used in (passive) seismology is the **miniseed** format.

It is a binary format containing both the ground motion records and some metadata (station code, date, sampling step, etc.).

To read and analyze these data in a Python environment, you will use the **obspy** library. It is a relatively complete suite of tools that allows you to perform a whole set of processing of seismological data (in different data formats including *Miniseed*). 

An exhaustive documentation and many examples of use of Obspy are available here: https://github.com/obspy/obspy/
You can in particular watch the tutorials : https://docs.obspy.org/tutorial/ and https://krischer.github.io/seismo_live_build/tree/index.html 

In this notebook, we only cover some basic commands that will be useful to read, manipulate, represent and do some simple analysis of seismological data. You will need all those simple tools for the follwoing lectures and praticals. 

---------
**Obspy Installation**

If you wish to use *Obspy* on your personal computers, you must follow the installation procedure available here: https://github.com/obspy/obspy/wiki#installation
The easiest way is to have a dedicated virtual environment via *Anaconda*.

-----------

## Importations of modules

In the cell below we load the modules of *Obspy* useful later

In [ ]:
from obspy import read, UTCDateTime, read_inventory

## Reading data in miniseed format

The following cell shows you how to read a miniseed data file via *Obspy* with the `read` method

Here we read the file *AIS.G.00.BHZ.2020.103.mseed* corresponding to the 
- station AIS from the network G (GEOSCOPE)
- for the data sequence starting on day 103 in 2020 with a duration of 1 day
- for the vertical component (*BHZ*)

Be careful to specify the right path to your file!

In [ ]:
path_data = 'DATA/'

S = read(path_data + 'AIS.G.00.BHZ.2020.103.mseed')

print(S)

#### *Stream* and *Trace* objects

As the `print(S)` command indicates, `S` is an object of type *Stream*.

Such a *Stream* object can be thought of as a list that can contain multiple *Trace* objects. A *Trace* corresponds to a continuous recording on a channel (*Z*, *N* or *E*) at a single sensor.

Here there is only one *Trace* in the `S` object.
To extract the corresponding *Trace*, we perform:

In [ ]:
T = S[0]

print(T)
print(type(T))

#### Metadata associated with a *Trace*

The metadata corresponding to this *Trace* are available via the *.stats* method:

In [ ]:
print(T.stats)
print('------')
fs = T.stats.sampling_rate # sampling frequency
print('fs : ',fs)

#### Note on dates in Obspy

Dates (e.g. `starttime` for the first sample of the *Trace*) are expressed via objects of type *UTCDateTime*.

We can declare such dates via the method `UTCDateTime` (*be careful to respect lowercase/uppercase*) with as arguments: year,month,day,[hour],[minute],[second].

The basic unit for objects of type *UTCDateTime* is the second.


In [ ]:
t0 = T.stats.starttime

print('t0 trace : ',t0)
print(type(t0))

time = UTCDateTime(2020,4,12,0,0,0)
time2 = time + 120
print('time : ',time)
print('time + 120s : ',time2)

#### Access and modification of the time series

The data vector is available via the `.data` attribute of the *Trace* object, this data vector is of type *numpy.array*.

In [ ]:
print(T.data[0:100])

#### Instrument response correction

As you can see in the output in the above cell, the data vector is composed of *raw data* which are intergers and not ground velocity in m/s. This raw data unit is call *counts*. 

In seismology it is almost always necessary to correct the raw data in *counts* from the instrument response of the sensor in order to convert the data to usuful physical units. Indeed, as any recording device, seismometers perturb the measurements. For any given seismic station, the network operator needs to provide the instrument response (i.e., the transfer function) of the station - this is usually an `.xml` file. The users can than correct the raw data from this instrument response and convert back the data in actual m/s.

In `obspy` this step can be easely done with the `.remove_response` attribute of the *Trace* object. We first have to read the instrument response in `.xml` format and then use this inventory to construct the tranfer function and perform the correction. Note that we apply a pre_filt which is simply a filter  


In [ ]:
inv = read_inventory(path_data + "AIS.xml")
pre_filt = [0.001, 0.005, 9, 9.9]
T.remove_response(inventory=inv, pre_filt=pre_filt, output="VEL",
                   water_level=60, plot=True)

## Main methods of *Trace* (and *Stream*) objects

All the interest of *Obspy* is to have methods/attributes already written allowing a visualization and a fast treatment of objects of type *Trace* or *Stream*.

In the following we will illustrate some of them. 
To access all the possible methods you just have to write in a code cell the name of the variable (for example `S`) followed by a dot (`S.`) and press the *tabulation* key. Or use the command `dir(S)`.

In [ ]:
dir(S)

####  Graphical representation 

This is the `.plot` method which has many options (see https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.plot.html)

In [ ]:
T.plot()

But for more "freedom", you can of course use directly *matplotlib.pyplot* from the `.data` vector

the *Trace* object has a `.times` method which simply allows you to get a vector (*numpy.array*) with the appropriate time (see the different options)

In [ ]:
y = T.data
t = T.times(type='relative')

print(y[0:20])
print('----')
print(t[0:20])

plt.figure(figsize=(12,4))
plt.plot(t,y)
plt.xlabel('relative time')
plt.ylabel('amplitude (m/s)')
plt.grid()

#### Cutting a portion of data

For this there are 2 possibilities:  
- the `trim` method which will modify the original *Trace* permamente ("in place")- **Warning !**
- the `slice` method whose result must be saved in another variable (the original *Trace* is not modified)

Each time you have to specify the start and end date of the segment you want to cut.

For example to extract a 1 hour (3600s) record starting 6.5 hours (6.5*3600 s) after the time of the first sample:

In [ ]:
t0 = T.stats.starttime
t1 = T.stats.starttime + (6.5*3600)

# Option 1 (we modify the original trace)
T.trim(t1,t1+3600)
print(T)
T.plot()

# Option 2 (sans toucher à la trace originale)
#T2 = T.slice(t1,t1+3600)
#print(T)
#print(T2)
#T2.plot()

#### Removal of the average and appodization of a *Trace*.

*Both of these steps must be performed before filtering a *Trace**.

To remove the average of a *Trace* use the `.detrend` method (which can also remove a linear trend)

To perform a matching (by default with a Hanning window), use the `.taper` method, indicating as parameter the % of the trace that will be matched at the beginning and at the end of the window.

**Warning**: the original trace is definitely modified ("in place") by these operations!
If you want to keep the original trace you have to copy it first with `Tkeep = T.copy()` (`Tkeep` is the preserved version of `T` even if `T` is modified afterwards)

In [ ]:
T.detrend("demean")
T.taper(0.1)

T.plot()

#### Filtering a *Trace

We use the `.filter` method.
This method applies a butterworth filter (others available!) of type *lowpass*, *highpass* or *bandpass* (1st argument) and then specifying the `freqmin` and/or `freqmax` arguments

The example below performs a bandpass filtering between 0.01 and 1 Hz

In [ ]:
Tkeep = T.copy()

T.filter("bandpass",freqmin=.01,freqmax=1)

T.plot()

#### Spectrogramme of a trace

To get a quick graphical representation of the spectrogram of a *Trace* you can use the `.spectrogram` method.

See the many possible options here: https://docs.obspy.org/packages/autogen/obspy.imaging.spectrogram.spectrogram.html#obspy.imaging.spectrogram.spectrogram

Here we apply it to the *Trace* `Tkeep` (because the *Trace* `T` has been modified by the filtering) with a log scale in frequency and amplitude and for windows of 30s (`wlen`) overlapping by 50% (`per_lap`)


In [ ]:
Tkeep.spectrogram(log=True,dbscale=True,wlen=30,per_lap=0.5,clip=[0.5,1])

However, it is recommended to use the `spectrogram` method of the `scipy.signal` module (cf. https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html) which will be applied directly to the data (`Tkeep.data` attribute) in order to have direct access to the variables allowing to manipulate the spectrogram and to represent it as we wish.

In [ ]:
from scipy.signal import spectrogram

fs = Tkeep.stats.sampling_rate
nperseg = int(30*fs)
noverlap = int(nperseg*0.5)
f,t,Sxx = spectrogram(Tkeep.data, fs=fs, nperseg=nperseg, noverlap=noverlap)

Sxx = 10*np.log10(Sxx)

In [ ]:
plt.figure(figsize=(12,8))
plt.pcolor(t,f,Sxx)
plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')
plt.colorbar()
plt.clim([-180,-50])
plt.ylim((.05,10))
plt.yscale('log')

## Reading of several data files + selection and merge

You can read multiple data files directly with `read` :

In [ ]:
S = read(path_data + "*SCZ*")
print(S)

In this example we see that the *Stream* object contains 5 *Trace* objects.

We can extract only one or several *Trace* objects thanks to the `.select` method by filtering on the network (here *G*), station (here *SCZ*) or component (here *BHZ*, *BHN* or *BHE*) codes

For example to extract only the *Trace* corresponding to the east component


In [ ]:
SE = S.select(station="*",channel='BHE')

print(SE)

We see in this (very) particular case that for the *E* component there are 3 *Traces* (there were 3 files for this component).

These 3 traces overlap over a large part of the recording as indicated by the representation

In [ ]:
SE.plot()

We can also visualize this better by plotting each traces with the same axis

In [ ]:
SE.sort(['starttime'])
fig = plt.figure()
for i in range(3):
    ax = fig.add_subplot(3, 1, i+1)
    t = np.linspace(SE[i].stats.starttime.timestamp - dt,
                    SE[i].stats.endtime.timestamp - dt,
                    SE[i].stats.npts)
    ax.plot(t, SE[i].data)
    plt.xlim(0, 5000)
plt.show()

With the `.merge` method it is possible to merge these 3 traces into one (in case the common part of these 3 traces is similar), and interpolate the elements if gaps are present. You can see all options here: https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.merge.html#obspy.core.stream.Stream.merge

In [ ]:
SE.merge(method=1)

print(SE)

SE.plot()

--------
NB: Objects of type *Stream* are **iterable**, i.e. you can easily perform a `for` loop on each *Trace* of a *Stream* to apply treatments to it.

For example:

In [ ]:
S = read(path_data + '*SCZ*')
SE = S.select(station="*",channel='BHE')
for T in SE:
    T.detrend("demean")
    T.taper(0.1)
    T.filter("bandpass",freqmin=0.01,freqmax=0.1)
    T.plot()

------


## Writing new files in miniseed format

Once you have performed various treatments (slicing, filtering, ...) you can save your new *Trace* in a file in miniseed format via the `.write` method

In [ ]:
fic_out = 'toto.mseed'
T.write(fic_out,format="MSEED")